In [3]:
import numpy as np
import torch
import torch.nn as nn
from vae_original import *
import gym

In [4]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

EasyProcessError: start error <EasyProcess cmd_param=['Xvfb', '-help'] cmd=['Xvfb', '-help'] oserror=[Errno 2] No such file or directory: 'Xvfb': 'Xvfb' return_code=None stdout="None" stderr="None" timeout_happened=False>

In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) # error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [2]:
USE_CUDA = True
use_cuda = USE_CUDA and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print('Using device', device)
import multiprocessing
NUM_WORKERS = multiprocessing.cpu_count()
print('num workers:', NUM_WORKERS)

Using device cpu
num workers: 4


In [3]:
vae = VAE(device)
vae.to(device, dtype=torch.float)

VAE(
  (encoder): Sequential(
    (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2))
    (5): ReLU()
    (6): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2))
    (7): ReLU()
  )
  (mufc): Linear(in_features=1024, out_features=32, bias=True)
  (logvarfc): Linear(in_features=1024, out_features=32, bias=True)
  (decoder_fc): Linear(in_features=32, out_features=1024, bias=True)
  (decoder): Sequential(
    (0): ConvTranspose2d(1024, 128, kernel_size=(5, 5), stride=(2, 2))
    (1): ReLU()
    (2): ConvTranspose2d(128, 64, kernel_size=(5, 5), stride=(2, 2))
    (3): ReLU()
    (4): ConvTranspose2d(64, 32, kernel_size=(6, 6), stride=(2, 2))
    (5): ReLU()
    (6): ConvTranspose2d(32, 3, kernel_size=(6, 6), stride=(2, 2))
    (7): Sigmoid()
  )
)

In [4]:
vae_path = './vae_original.pt'
vae.load_state_dict(torch.load(vae_path, map_location=torch.device(device)))

<All keys matched successfully>

In [5]:
from rlhw_util import * # <-- look whats inside here - it could save you a lot of work!

def run_iteration(mode, N, agent, gen, horizon=None, render=False):
    train = mode == 'train'
    if train:
        agent.train()
    else:
        agent.eval()

    states, actions, rewards = zip(*[gen(horizon=horizon, render=render) for _ in range(N)])

    loss = None
    if train:
        loss = agent.learn(states, actions, rewards)

    reward = sum([r.sum() for r in rewards]) / N

    return reward, loss

In [13]:
env_name = 'CarRacing-v0'
e = Pytorch_Gym_Env(env_name)
#state_dim = e.observation_space.shape[0]
#action_dim = e.action_space.n
state_dim = e.observation_space.shape #  (96 x 96 x 3)
action_dim = e.action_space.shape  # there are 3 actions

In [ ]:
# action space is 3 triples: (s, a, d): ([-1, 1], [0, 1], [0, 1])
# steering (hard left, hard right)
# acceleration (none to full acceleration)
# decceleration (none to full on braking)

In [ ]:
class RNN(nn.Module):
    def __init__(self, batch_size=1, a_size=3, z_size=32, h_size=256, num_layers=1):
        super(LSTM,self).__init__()
        
        self.batch_size = batch_size
        self.h_size = h_size

        #self.rnn = nn.LSTM(input_size=a_size + z_size, hidden_size=256, num_layers=num_layers)
        self.rnn = nn.LSTMCell(input_size=a_size + z_size, hidden_size=256, num_layers=num_layers)
        
    def forward(self, a, z, h_curr, c_curr):
        az = torch.cat((a, z), dim=1)
        h_next, c_next = self.rnn(az, (h_curr, c_curr))
        return (h_next, c_next)

In [ ]:
class MDN_RNN(nn.Module):
    def __init__(self, z_dim, h_dim, action_dim):
        super(Controller, self).__init__()
        self.c = nn.Linear(z_dim + h_dim, action_dim)
                
    def forward(self, z, h):
        full_state = torch.cat((z, h), dim=1)
        return self.c(full_state)
    
    def get_action(self, z, h):
        with torch.no_grad():
            full_state = torch.cat((z, h), dim=1)
            return self.c(full_state)

In [21]:
class Controller(nn.Module):
    def __init__(self, z_dim, h_dim, action_dim=3):
        super(Controller, self).__init__()
        self.c = nn.Linear(z_dim + h_dim, action_dim)
                
    def forward(self, z, h):
        full_state = torch.cat((z, h), dim=1)
        return self.c(full_state)
    
    def get_action(self, z, h): # dont think this function is needed
        with torch.no_grad():
            full_state = torch.cat((z, h), dim=1)
            return self.c(full_state)

In [ ]:
gen = Generator(e, agent)

In [ ]:
train_reward, train_loss = run_iteration('train', num_train, agent, gen)
    eval_reward, _ = run_iteration('eval', num_eval, agent, gen)